In [ ]:
!nvidia-smi
!pip install happytransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Summarizer")

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PegasusForConditionalGeneration.from_pretrained("pegasus/checkpoint-10000").to(torch_device)
tokenizer = PegasusTokenizer.from_pretrained('tuner007/pegasus_summarizer')

def get_summarizer(input_text, temperature):
  batch = tokenizer(input_text,truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(torch_device)
  gen_out = model.generate(**batch,do_sample=True,max_length=128,num_beams=5, num_return_sequences=1, temperature=temperature)
  output_summarizer = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_summarizer

In [ ]:
import pandas as pd
data = pd.read_csv("test_data_1000.csv")
# data.drop(columns=['id'], inplace=True)
context = data['context']

In [ ]:
# Run model
outputs_peg_summarizer = []
outputs_peg_summarizer_t = []
i=1

for cont in context:
  print(i)
  i+=1
  cont = "summarize: " + cont
  outputs_peg_summarizer.append(get_summarizer(cont,1.5)[0])
  outputs_peg_summarizer_t.append(get_summarizer(cont,0.8)[0])

In [ ]:
# Save File
data['summarizer_1.5'] = outputs_peg_summarizer
data['summarizer_0.8'] = outputs_peg_summarizer_t
data.to_csv("summarizer.csv", index=False)